# Algorithmic Motion Planning (236610)
John Noonan and Eli Shafer<br>
Homework 2<br>
December 2019

### Warmup

Question 1: In a 2-dimensional unit ball (namely, a disk), how much of the volume is located at most 0.1 units from the surface?

**Answer: C Roughly 20%**

Question 2: In a 9-dimensional unit ball, how much of the volume is located at most 0.1 units from the surface?

**Answer: Roughly 60%**

### Exercise

Define the fraction of the volume that is $\epsilon$ distance from the surface of a d-dimensional unit ball as $n_d(\epsilon)$. 

**Plot** $\eta_d(\epsilon)$ as a function of $d$ for $\epsilon = 0.2, 0.1, 0.01$ for $d = 2 ... 10$

![plot_of_volume](ex_plot.png)

**Discuss** the implications to reducing the connection radius required for a sampling-based algorithm to maintain asymptotic optimality.

*This discussion is related to what we saw in class regarding the proof of probabilistic completeness for the PRM algorithm.  When proving asymptotic optimality for sampling-based algorithms, we typically take the path, cover it with balls where the radius of each ball is proportional to the clearance, and argue that within each ball, with high probability, we will get a sample and can connect two balls which will yield a free path since our connection radius is big enough.  It's important to note that the probabilistic argument depends on the volume of each one of the balls, and the notion of connection is critical because if the connection radius is too small, then there will be points in each ball but we **won't be able to connect them**.  In other words, if the connection radius is reduced too much then points will not be connected, nullifying our conclusions for asymptotic optimality.  On the other hand, if the connection radius is too large, then there will be too many points. Therefore, as we discussed, there has been a quest to determine the minimal connection radius or minimal expected number of nearest neighbors which are needed to determine certain desirable algorithmic properties.*

### 3 Motion Planning: Search and Sampling

#### 3.3 A* Implementation